# Task 3 - Experience Analytics

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file

In [ ]:
load_dotenv('../venv/venv')

# Retrieve database connection details from environment variables

In [ ]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
table_name = 'xdr_data'

# Create the connection string

In [ ]:
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Query the data

In [ ]:
query = 'SELECT * FROM xdr_data'
data = pd.read_sql(query, engine)

# Display the data

In [ ]:
print(data.head())

In [ ]:
print(data.info())

# Aggregate metrics per customer

In [ ]:
# Aggregate per customer
aggregated_data = data.groupby('IMSI').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]
}).reset_index()

# Calculate total TCP retransmission, RTT, and throughput
aggregated_data['Total_TCP'] = (aggregated_data['TCP DL Retrans. Vol (Bytes)'] + aggregated_data['TCP UL Retrans. Vol (Bytes)'])
aggregated_data['Total_RTT'] = (aggregated_data['Avg RTT DL (ms)'] + aggregated_data['Avg RTT UL (ms)'])
aggregated_data['Total_Throughput'] = (aggregated_data['Avg Bearer TP DL (kbps)'] + aggregated_data['Avg Bearer TP UL (kbps)'])

# Drop the intermediary columns
aggregated_data = aggregated_data.drop(columns=[
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)'
])

aggregated_data.head()

# Compute and list 10 of the top, bottom, and most frequent

In [ ]:
def compute_top_bottom_frequent(data, column, top_n=10):
    top_values = data[column].nlargest(top_n).reset_index(drop=True)
    bottom_values = data[column].nsmallest(top_n).reset_index(drop=True)
    most_frequent_values = data[column].mode()
    
    # If there are fewer than top_n most frequent values, adjust
    if len(most_frequent_values) > top_n:
        most_frequent_values = most_frequent_values.head(top_n)
    
    return {
        'top': top_values,
        'bottom': bottom_values,
        'most_frequent': most_frequent_values
    }

# Compute statistics for each metric
tcp_stats = compute_top_bottom_frequent(aggregated_data, 'Total_TCP')
rtt_stats = compute_top_bottom_frequent(aggregated_data, 'Total_RTT')
throughput_stats = compute_top_bottom_frequent(aggregated_data, 'Total_Throughput')

# Function to print the results in a readable format
def print_stats(stats, title):
    print(f"\n{title}")
    print("\nTop 10 values:")
    print(stats['top'])
    print("\nBottom 10 values:")
    print(stats['bottom'])
    print("\nMost Frequent values:")
    print(stats['most_frequent'])

# Print the results for each metric
print_stats(tcp_stats, "Total TCP Stats")
print_stats(rtt_stats, "Total RTT Stats")
print_stats(throughput_stats, "Total Throughput Stats")